In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from wildfires.analysis import *
from wildfires.data.datasets import *

In [ ]:
new = NewERA5_DryDayPeriod()

In [ ]:
old = ERA5_DryDayPeriod()
old.cubes = iris.cube.CubeList([old.cube[:20]])

In [ ]:
iris.cube.CubeList([new.cube, old.cube]).realise_data()

In [ ]:
diff = new.cube.data - old.cube.data

In [ ]:
rel_abs_diff = np.mean(np.abs(diff) / old.cube.data, axis=0)

In [ ]:
rel_diff = np.mean(diff / old.cube.data, axis=0)

In [ ]:
cube_plotting(new.cube, fig=plt.figure(figsize=(15, 7)), log=True)

In [ ]:
cube_plotting(old.cube, fig=plt.figure(figsize=(15, 7)), log=True)

In [ ]:
cube_plotting(rel_abs_diff, fig=plt.figure(figsize=(15, 7)))

In [ ]:
cube_plotting(rel_diff, cmap_midpoint=0, fig=plt.figure(figsize=(15, 7)))

In [ ]:
np.where(rel_diff == np.min(rel_diff))

In [ ]:
new.cube.coord("latitude").points[449], new.cube.coord("longitude").points[837]

In [ ]:
plt.hist(diff.flatten(), bins=1000)
plt.yscale("log")

In [ ]:
import glob
import os

from tqdm import tqdm

tpdir = os.path.join(DATA_DIR, "ERA5", "tp_daily")

# Sort so that time is increasing.
filenames = sorted(
    glob.glob(os.path.join(tpdir, "**", "*_daily_mean.nc"), recursive=True)
)

precip_cubes = iris.cube.CubeList()

prev_dry_day_period = None
prev_end = None

with warnings.catch_warnings():
    warnings.filterwarnings(
        "ignore",
        message=(
            "Collapsing a non-contiguous coordinate. Metadata may not "
            "be fully descriptive for 'time'."
        ),
    )
    for filename in tqdm(filenames[:20]):
        raw_cube = iris.load_cube(filename)
        precip_cubes.append(raw_cube)

In [ ]:
precip_cubes = homogenise_cube_attributes(precip_cubes)

In [ ]:
all_combined = precip_cubes.concatenate_cube()

In [ ]:
iris.cube.CubeList([all_combined]).realise_data()

In [ ]:
combined = all_combined.intersection(latitude=(22.25, 22.26), longitude=(29.25, 29.26))

In [ ]:
N = 400
plt.figure(figsize=(20, 8))
plt.plot(combined.data.flatten()[:N], marker="o", linestyle="")
plt.hlines(y=M_PER_HR_THRES, xmin=0, xmax=N)

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(
    old.cube.intersection(
        latitude=(22.25, 22.26), longitude=(29.25, 29.26)
    ).data.flatten()[: N // 30],
    marker="o",
    linestyle="",
)

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(
    new.cube.intersection(
        latitude=(22.25, 22.26), longitude=(29.25, 29.26)
    ).data.flatten()[: N // 30],
    marker="o",
    linestyle="",
)

In [ ]:
np.where(rel_diff == np.max(rel_diff))

In [ ]:
all_combined.shape, old.cube.shape, new.cube.shape

In [ ]:
old.cube.coord("latitude").points[403]

In [ ]:
old.cube.coord("longitude").points[660]

In [ ]:
plt.figure(figsize=(20, 8))
data = all_combined.intersection(latitude=(10.75, 10.76), longitude=(-15, -14.9)).data
max_d = np.max(data)
below = data < M_PER_HR_THRES
plt.scatter(
    list(range(len(data))), data, marker="o", c=["r" if b else "b" for b in below]
)
plt.hlines(y=M_PER_HR_THRES, xmin=0, xmax=all_combined.shape[0])
x = 0
for cube in precip_cubes:
    d = cube.shape[0]
    plt.vlines(x=[x, x + d], ymin=0, ymax=max_d, colors="g")
    x += d

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(old.cube.data[:, 403, 660], marker="o", linestyle="")

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(new.cube.data[:, 403, 660], marker="o", linestyle="")

In [ ]:
import scipy.ndimage

# Find contiguous blocks in the time dimension where dry_days is True.
structure = np.zeros((3,), dtype=np.int64)
structure[:] = 1
labelled = scipy.ndimage.label(below, structure=structure)
slices = scipy.ndimage.find_objects(labelled[0])

In [ ]:
labelled

In [ ]:
slices